In [ ]:
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt

# from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
%matplotlib widget
plt.rcParams['figure.figsize'] = [8,6]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

# The emergence of band structures

If you have questions about this notebook, e-mail Oleg Malanyuk (oleg.malanyuk@epfl.ch) 

This jupyter notebook is intended to provide a illustration of the band structures in solides, starting from the points of view of free electrons and atomic orbitals. The goals are:
- To understand how the Bloch theorem leads to band structures.
- To see how the electronic bands emerge from the interplay of periodic potential and kinetic energy.
- Understanding the relationship between the Schrodinger equation and the calculation*

## The Bloch theorem
In solids, the lattice has discrete translational symmetry. The Bloch theorem says that, such symmetry applies to the wavefunction as well, such that $$[H,R_i]=0$$
To study the energy levels in solides, one can adapt the simplest Hamiltonian:$$ H = -\frac{h^2}{2m} \nabla^2 + U(r)$$ where $U(r+R_i)=U(r)$   



From the commutation relations of the Hamiltonian, the wavefunctions should be of the form $\psi(nR_i + r) = \psi(r) \exp(-inR_i)$ which is called Bloch functions.

## From the atomic orbitals/free electrons to band structures

The Hamiltonian $$ H = -\frac{h^2}{2m} \nabla^2 + U(r)$$ contains two components: the kinetic energy $E_k=-\frac{h^2}{2m} \nabla^2$ and the potential $U$. Thus, we can look into the formation of band structure from the limits of free electrons($E_k >> U$) or localized orbitals($E_k<<U$) and treating the other term as a perturbation.

In [ ]:
t_g = 0.5
u_g = 0.5
V_type_g = 'Sine Wave'

def interactive_bands_V(V_type):
    '''
    :param k: wave number
    '''
    global t_g, u_g, V_type_g 
    global ax1, ax2, ax3
    V_type_g = V_type
    t = t_g
    u = u_g
    nq=30
    n = 30
    fs = 500
    # Define the position of the original lattice
    qs=np.arange(nq)
    plx=np.zeros((nq+1,nq+1))
    ply=np.zeros((nq+1,nq+1))
    mat=np.zeros((nq+1,nq+1))
    x = np.linspace(-n, n, n*fs+1, endpoint=True)
    
    if (V_type == 'Sine Wave'):
        V = -np.cos(x*2*np.pi)
        for i in range(nq+1):
            U=np.ones(nq+1)*(-u)
            K=np.ones(nq+1)
            for j in range(nq+1):
                K[j] = t*(i/nq - 1/2 + j - nq/2)**2
            mat=np.zeros((nq+1,nq+1))
            mat=mat+np.diag(U,-1)[:-1,:-1]+np.diag(U,1)[:-1,:-1]+np.diag(K,0)
            ply[i]=np.linalg.eigvalsh(mat)
            plx[i]=np.ones(nq+1)*(i-nq/2)/nq

    elif (V_type == 'Series of Delta Functions'):
        V = u*(np.ones(x.shape) - np.ceil(x-np.floor(np.round(x,4))))
        for i in range(nq+1):
            K=np.ones(nq+1)
            for j in range(nq+1):
                K[j] = t*(i/nq - 1/2 + j - nq/2)**2
            mat=np.ones((nq+1,nq+1))*(u)
            mat=mat+np.diag(K,0)

            ply[i]=np.linalg.eigvalsh(mat)#+t*2
            plx[i]=np.ones(nq+1)*(i-nq/2)/nq
            
    elif (V_type == 'Square Wave'):
        V = 2*np.round((-np.cos(x*2*np.pi)+1)/2, 0) - np.ones(x.shape)
        for i in range(nq+1):
            for j in range(nq+1):
                if ((i - j) %2!=0):
                    mat[i][j] = (-1)**(int((i-j+1)/2))*u/(i-j)
        for i in range(nq+1):
            mat_sq = np.copy(mat)
            K=np.ones(nq+1)
            for j in range(nq+1):
                K[j] = t*(i/nq - 1/2 + j - nq/2)**2
            mat_sq=mat_sq+np.diag(K,0)
            ply[i]=np.linalg.eigvalsh(mat_sq)#+t*2
            plx[i]=np.ones(nq+1)*(i-nq/2)/nq
    
    ax1.clear()
    ax1.set_xlabel('k vector (as fraction of G)');
    ax1.set_ylabel('Electron Energy');
    ax1.set_title('Band Structure')
    
    for i in range(0,nq+1):
        ax1.plot(plx[:,i],ply[:,i],'b')
        ax1.plot(plx[:,i]+1,ply[:,i],'g')
        ax1.plot(plx[:,i]-1,ply[:,i],'r')
        #plt.plot(plx[:,i]-nq*2,ply[:,i])
    Vq = np.fft.fft(V)
    k = np.fft.fftfreq(n = x.shape[0], d = 2/fs)

    
    ax2.clear()
    ax2.set_xlabel('Distance x');
    ax2.set_ylabel('Potential U(x)');
    ax2.set_title('Shape of Potential U(x)')
    ax2.set_yticks(ticks=[-1,0,1]);
    ax2.plot(x[int((n/2-1/2)*fs)-2:int((n/2+1/2)*fs)+2], V[int((n/2-1/2)*fs)-2:int((n/2+1/2)*fs)+2])
    
    ax3.clear()
    ax3.plot(k, Vq.real)
    ax3.set_xlim(0,10)
    ax3.set_xticks(ticks=np.arange(0, 10, step=1));
    ax3.set_yticks(ticks=[0]);
    ax3.set_title('Fourier transform of Potential U(q)')
    ax3.set_xlabel('q vector (as fraction of G)');
    ax3.set_ylabel('Relative Intensity of Uq');
    ax3.grid
    
    x = np.linspace(-1.5*nq,1.5*nq,100)
    plz=t*(x/(nq))**2 + np.ones(x.shape[0])*ply[int(nq/2+1)][0]
    x = x/nq
    ax1.plot(x,plz,'y')
    ax = plt.gca()
    ax1.set_ylim(ply[int(nq/2+1)][0],(ply[int(nq/2+1)][7]+ply[int(nq/2+1)][8])/2)
    zz=ply[0,0]+2*t

    plt.show()

In [ ]:
def interactive_bands_tu(t,u):
    '''
    :param k: wave number
    '''
    global t_g, u_g, V_type_g
    global ax1, ax2, ax3
    t_g = t
    u_g = u
    V_type = V_type_g
    nq=30
    # Define the position of the original lattice
    qs=np.arange(nq)
    plx=np.zeros((nq+1,nq+1))
    ply=np.zeros((nq+1,nq+1))
    mat=np.zeros((nq+1,nq+1))
    
    if (V_type == 'Sine Wave'):
        for i in range(nq+1):
            U=np.ones(nq+1)*(-u)
            K=np.ones(nq+1)
            for j in range(nq+1):
                K[j] = t*(i/nq - 1/2 + j - nq/2)**2
            mat=np.zeros((nq+1,nq+1))
            mat=mat+np.diag(U,-1)[:-1,:-1]+np.diag(U,1)[:-1,:-1]+np.diag(K,0)
            ply[i]=np.linalg.eigvalsh(mat)#+t*2
            plx[i]=np.ones(nq+1)*(i-nq/2)/nq

    elif (V_type == 'Series of Delta Functions'):
        for i in range(nq+1):
            #U=np.ones(nq+1)*(-u)
            K=np.ones(nq+1)
            for j in range(nq+1):
                K[j] = t*(i/nq - 1/2 + j - nq/2)**2
            mat=np.ones((nq+1,nq+1))*(u)
            mat=mat+np.diag(K,0)


            ply[i]=np.linalg.eigvalsh(mat)#+t*2
            plx[i]=np.ones(nq+1)*(i-nq/2)/nq
            
    elif (V_type == 'Square Wave'):
        for i in range(nq+1):
            for j in range(nq+1):
                if ((i - j) %2!=0):
                    mat[i][j] = (-1)**(int((i-j+1)/2))*u/(i-j)
        for i in range(nq+1):
            mat_sq = np.copy(mat)
            K=np.ones(nq+1)
            for j in range(nq+1):
                K[j] = t*(i/nq - 1/2 + j - nq/2)**2
            mat_sq=mat_sq+np.diag(K,0)
            ply[i]=np.linalg.eigvalsh(mat_sq)#+t*2
            plx[i]=np.ones(nq+1)*(i-nq/2)/nq

    ax1.clear()
    ax1.set_title('Band Structure')
    ax1.set_xlabel('k vector (as fraction of G)');
    ax1.set_ylabel('Electron Energy');
    for i in range(0,nq+1):
        ax1.plot(plx[:,i],ply[:,i],'b')
        ax1.plot(plx[:,i]+1,ply[:,i],'g')
        ax1.plot(plx[:,i]-1,ply[:,i],'r')

    x = np.linspace(-1.5*nq,1.5*nq,100)
    plz=t*(x/(nq))**2 + np.ones(x.shape[0])*ply[int(nq/2+1)][0]
    x = x/nq
    ax1.plot(x,plz,'y')
    ax1.set_ylim(ply[int(nq/2+1)][0],(ply[int(nq/2+1)][7]+ply[int(nq/2+1)][8])/2)

    plt.show()

## An interactive look

In the following block, there is a simple example on how band structure emerges in lattices. The potential function $U$ is for a lattice with atomic positions $x_i \in \mathbb{Z}$. $U$ takes one of three forms: 

1. Sine wave: $U(x)=-2u*cos(2\pi x)$, 

2. Series of Delta functions centred on atomic positions: $U(x)=\sum_{x_i} u*\delta(x - x_i)$ 

3. Square wave: $U(x)= -u$ for $x_i - \frac{1}{2} < x \le x_i + \frac{1}{2}$ and $U(x)= u$ otherwise

Meanwhile, t regulates kinetic energy contribution (i.e. a free electron with momentum k would have energy $T=t*k^2$) 




By tuning the parameter t and u, one can travel between the periodic free electronic limit and the discrete energy levels. One can see that, at the limit $t<<u$, the spectrum resembles a series of discrete energy levels, which are rarely dependent on the wave vector $k$. Such a spectrum reflect the fact that the eigenstates are mostly localized by the strong potential. On the other hand, if we take $t$ as the dominant term, the spectrum then looks like a series of quadratic functions. This is consistent with the fact that the system is periodic and the electrons are nearly free. 

In between the two cases, there then arrives our band structures: the spectrum forms several energy bands while each band has some dispersion. 

In [ ]:
plt.clf
figs = plt.figure();
ax1 = plt.subplot(1,9,(1,3));

ax2 = plt.subplot(3,9,(6,9));

ax3 = plt.subplot(2,9,(15,18));


interact(interactive_bands_tu,t=widgets.FloatSlider(min=0.05, max=1.00, step=0.05, value = t_g),u=widgets.FloatSlider(min=0.00, max=1.00, step=0.05, value = u_g));
interact(interactive_bands_V, V_type=widgets.RadioButtons(
    value= V_type_g, 
    options=['Sine Wave', 'Series of Delta Functions', 'Square Wave'], 
    description='Potential:'
));